# 1. Template
最基本的用法極度類似 Python 的 String formatting，幫助開發者簡便地寫出格式一樣但內容不同的 prompt

## 1.1 LLM 用 Prompt

In [2]:
from langchain import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["person", "spirit"], template="{person}的替身是{spirit}。"
)

prompt1 = prompt_template.format(person="空條承太郎", spirit="白金之星")
print("Prompt 1:", prompt1)

prompt2 = prompt_template.format(person="喬瑟夫·喬斯達", spirit="隱者之紫")
print("Prompt 2:", prompt2)

Prompt 1: 空條承太郎的替身是白金之星。
Prompt 2: 喬瑟夫·喬斯達的替身是隱者之紫。


## 1.2 Chat Model 用 Prompt

當使用 Chat Model 時，最好是使用 `ChatTemplate`。因為 Chat Model 如之前所介紹的，是有針對對話紀錄優化過的模型，因此利用 `ChatTemplate` 將內容轉化成對話紀錄的形式，理論上來說對對話對話模型來說是最好的。

In [1]:
from langchain.prompts import ChatPromptTemplate
from pprint import pprint

prompt_template = ChatPromptTemplate.from_messages(
   [ ("system", "你是一個喜歡玩捉迷藏的小孩"), 
     ("human", "藏好了嗎?"),
     ("ai", "藏好了~"),
     ("human", "{name}，找到你了!"), 
  ]
)

pprint(prompt_template.format_messages(name="面麻"))

[SystemMessage(content='你是一個喜歡玩捉迷藏的小孩'),
 HumanMessage(content='藏好了嗎?'),
 AIMessage(content='藏好了~'),
 HumanMessage(content='面麻，找到你了!')]


若沒有甚麼角色間對話的問題，也可以用單純的文字 (text) 來建立對話模板 (ChatPrompteTemplate)

In [2]:
prompt_template = ChatPromptTemplate.from_template('請依照下面的資訊「{context}」來回答問題「{question}」')
pprint(prompt_template.format_messages(context="你的社團學長倒了一杯水", question="這杯水可以燃燒嗎?"))

[HumanMessage(content='請依照下面的資訊「你的社團學長倒了一杯水」來回答問題「這杯水可以燃燒嗎?」')]


# 2. In-Context Learning
Langchain 也提供了面向 In-Context Learning 的 Template, 方便我們舉例和挑選例子

In [1]:
from typing import Any, Dict
import numpy as np
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

icl_template = FewShotPromptTemplate(
    prefix="你是一個在新宿的廢棄大樓營業的占卜師。",
    example_prompt=PromptTemplate(
        input_variables=["person", "cuisine", "answer"],
        template="請告訴我 {person} 喜歡吃什麼樣的{cuisine}。答案：{answer}",
    ),
    examples=[
        {"person": "Richard", "cuisine": "麵", "answer": "牛肉麵"},
        {"person": "Kris", "cuisine": "便當", "answer": "港式燒臘"},
    ],
    example_separator="\n\n",
    suffix="請告訴我 {person} 喜歡吃什麼樣的{cuisine}。答案：",
    input_variables=["person", "cuisine"],
)

prompt_icl = icl_template.format(person="Sherry", cuisine="甜點")
print(prompt_icl)

你是一個在新宿的廢棄大樓營業的占卜師。

請告訴我 Richard 喜歡吃什麼樣的麵。答案：牛肉麵

請告訴我 Kris 喜歡吃什麼樣的便當。答案：港式燒臘

請告訴我 Sherry 喜歡吃什麼樣的甜點。答案：


當我們需要從很多的 examples 裡面挑選時，我們可以用 example selector

In [2]:
class RandomSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    # Need to be implemented
    def add_example(self, example: Dict[str, str]) -> Any:
        return self.examples.append(example)

    # Need to be implemented
    def select_examples(self, input_variables: dict[str, str]) -> list[dict]:
        """Select which examples to use based on the inputs."""
        return np.random.choice(self.examples, size=2, replace=False)


icl_template.examples = None
icl_template.example_selector = RandomSelector(
    [
        {"person": "Richard", "cuisine": "麵", "answer": "牛肉麵"},
        {"person": "Kris", "cuisine": "便當", "answer": "港式燒臘"},
        {"person": "Sheng", "cuisine": "飲料", "answer": "珍珠奶茶"},
        {"person": "Lifu", "cuisine": "餅乾", "answer": "品客"},
    ]
)

icl_prompt1 = icl_template.format(person="Sherry", cuisine="甜點")
print("icl_prompt1:")
print(icl_prompt1)

print("\n========================\n")

icl_prompt2 = icl_template.format(person="Sara", cuisine="壽司")
print("icl_prompt2:")
print(icl_prompt2)

icl_prompt1:
你是一個在新宿的廢棄大樓營業的占卜師。

請告訴我 Sheng 喜歡吃什麼樣的飲料。答案：珍珠奶茶

請告訴我 Kris 喜歡吃什麼樣的便當。答案：港式燒臘

請告訴我 Sherry 喜歡吃什麼樣的甜點。答案：


icl_prompt2:
你是一個在新宿的廢棄大樓營業的占卜師。

請告訴我 Richard 喜歡吃什麼樣的麵。答案：牛肉麵

請告訴我 Sheng 喜歡吃什麼樣的飲料。答案：珍珠奶茶

請告訴我 Sara 喜歡吃什麼樣的壽司。答案：


In [3]:
# 還有很多現成的 example selectors !!!
from langchain.prompts.example_selector import (
    LengthBasedExampleSelector,
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
    NGramOverlapExampleSelector,
)